# version refactored into functions

provides list of lists (which can be fed into dataframe)
given by processing pdf file with table spanning several pages
extracting only transactions data (and begin/end konto balance)

In [1]:
import camelot
import pandas

In [2]:
def remove_whitespace(x):
    """remove all whitespaces from a given string"""
    if x is not None:
        x = x.split()
        x = ''.join(x)
    return x

In [3]:
def extract_transactions(tables_transactions):
    """
    list of processed pdf pages -> dataframe with table that contains them all
    get the list of all monthly transactions, clean the data
    """
    transactions_list = []
    for table in tables_transactions:
        trans = table.df.values.tolist()
        for row in trans:
            transactions_list.append(row)
    transactions = pandas.DataFrame(transactions_list)

    transactions = transactions.applymap(remove_whitespace)

    transactions.columns = ['info', 'date', 'out', 'in']

    transactions = transactions[transactions.date != 'Valuta']
    transactions = transactions[transactions.date != 'zu']
    transactions = transactions.fillna(value="")
    transactions.index = range(len(transactions.index))
    
    return transactions

In [4]:
def merge_transactions(transactions, begin_month_data, end_month_data):
    """
    dataframe of transactions -> list transactions (each is a list)
    merges 3 rows of one transaction into one, discard the rest
    """
    list_merged_transactions = []
    
    list_merged_transactions.append(begin_month_data)

    use_this_row = 0 # we'll use 3 consecutive rows, and merge them 
    for index, row in transactions.iterrows():
            if (row['date'] != ''):
                use_this_row = 1
                current_info = row['info']
                current_date = row['date']
                current_in = row['in']
                current_out = row['out']
            elif use_this_row > 0:
                use_this_row += 1
                if use_this_row <= 3: # unused row after row with date
                    current_info += ". . . ." + row['info']
                    if use_this_row == 3:
                        list_merged_transactions.append([current_info, current_date, current_out, current_in])
                        use_this_row = 0
    
    list_merged_transactions.append(end_month_data)
 
    return list_merged_transactions


figuring out column dimensions in the console
```
camelot -p 4 stream -plot text test.pdf &
camelot -p 4 stream -plot textedge test.pdf &
camelot -p 4 stream -plot grid test.pdf &
```

check in the tool where are the limits

final command

```
camelot --format html --output test.html --pages 4 stream -R 0,600,600,0 -C 290,330,460 test.pdf
```

In [5]:
def process_file(input_file):
    """ for a given path to a pdf file, extract the transactional data as a list of transactions (list)"""

    # table_areas = top_left x, y, bottom_right x, y // 0,0 is bottom left
    # read all pages in bottom 600x600 square
    tables_transactions = camelot.read_pdf(input_file, flavor='stream', pages='1-end',
                                          table_regions=['0,600,600,0'], columns=['290,330,460'])
    tables_for_balance = camelot.read_pdf(input_file, pages='1-end')

    trans_begin = tables_for_balance[0].df # take 3 lines
    begin_month_balance = ''.join(trans_begin.iat[1,0].split())
    begin_month_data = ["BEGIN " + begin_month_balance[21:28] + " = " + begin_month_balance[28:], "", "", ""]

    trans_end = tables_for_balance[-1].df # take 3 lines
    end_month_balance = ''.join(trans_end.iat[-2,0].split())
    end_month_data = ["END " + end_month_balance[21:28] + " = " + end_month_balance[28:], "", "", ""]
    
    return merge_transactions(extract_transactions(tables_transactions), begin_month_data, end_month_data)
    